In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from typing import Tuple, Union

# Convolution from scratch 

- Basic convolution: same size array and kernel
- Image Convolution: No stride and no padding
- Convolution with padding
- Convolution with stride and padding
- Multi - channel convlution

## Basic convolution: Same size array and kernel 

In [4]:
def conv_basic(input_array, kernel):
    """
    Most basic convolution: element-wise multiplication and sum
    Input array and kernel must have the same size
    
    Args:
        input_array: 2D numpy array
        kernel: 2D numpy array (same size as input_array)
    
    Returns:
        Single scalar value (sum of element-wise multiplication)
    """
    if input_array.shape != kernel.shape:
        raise ValueError(f"Input shape {input_array.shape} must match kernel shape {kernel.shape}")
    
    # Element-wise multiplication and sum
    result = np.sum(input_array * kernel)
    return result

Test the above defined function 

In [14]:

# Create simple 3x3 input and kernel
input_patch = np.array([[1, 2, 3],
                       [4, 5, 6], 
                       [7, 8, 9]])

kernel = np.array([[1, 0, -1],
                  [1, 0, -1],
                  [1, 0, -1]])  # Vertical edge detector

result = conv_basic(input_patch, kernel)

print("Input patch:")
print(input_patch)
print("\nKernel:")
print(kernel)
print(f"\nConvolution result: {result}")
print("This is just: sum of (input * kernel element-wise)")
print()

Input patch:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Kernel:
[[ 1  0 -1]
 [ 1  0 -1]
 [ 1  0 -1]]

Convolution result: -6
This is just: sum of (input * kernel element-wise)



## Image Convolution - No Stride, No Padding

In [8]:
def conv_image_basic(image, kernel):
    """
    Convolution on a 2D image with a smaller kernel
    No stride (stride=1), no padding
    
    Args:
        image: 2D numpy array (height, width)
        kernel: 2D numpy array (smaller than image)
    
    Returns:
        2D numpy array (convolved output)
    """
    img_h, img_w = image.shape
    ker_h, ker_w = kernel.shape
    
    # Calculate output dimensions
    # the complete formula is [(W−K+2P)/S]+1 
    # in this case, P = S = 0 
    out_h = img_h - ker_h + 1
    out_w = img_w - ker_w + 1
    
    # Initialize output
    output = np.zeros((out_h, out_w))
    
    # Slide kernel over image
    for i in range(out_h):
        for j in range(out_w):
            # Extract patch from image
            patch = image[i:i+ker_h, j:j+ker_w]
            # Use our basic convolution function
            output[i, j] = conv_basic(patch, kernel)
    
    return output

Test the above defined function, and compare it's results with pytorch implementations

In [15]:

# Create a simple 5x5 image (you can also use a real image) 
image = np.array([[1, 2, 3, 4, 5],
                 [6, 7, 8, 9, 10],
                 [11, 12, 13, 14, 15],
                 [16, 17, 18, 19, 20],
                 [21, 22, 23, 24, 25]])

kernel = np.array([[1, 0, -1],
                  [1, 0, -1],
                  [1, 0, -1]])

result = conv_image_basic(image, kernel)

print("Input image (5x5):")
print(image)
print(f"\nKernel (3x3):")
print(kernel)
print(f"\nOutput shape: {result.shape} (expected: {(5-3+1, 5-3+1)})")
print("Output:")
print(result)

# Verify with PyTorch
image_torch = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)  # Add batch and channel dims
kernel_torch = torch.from_numpy(kernel).float().unsqueeze(0).unsqueeze(0)
torch_result = F.conv2d(image_torch, kernel_torch).squeeze().numpy()

print(f"\nPyTorch result:")
print(torch_result)
print(f"Results match: {np.allclose(result, torch_result)}")
print()

Input image (5x5):
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]

Kernel (3x3):
[[ 1  0 -1]
 [ 1  0 -1]
 [ 1  0 -1]]

Output shape: (3, 3) (expected: (3, 3))
Output:
[[-6. -6. -6.]
 [-6. -6. -6.]
 [-6. -6. -6.]]

PyTorch result:
[[-6. -6. -6.]
 [-6. -6. -6.]
 [-6. -6. -6.]]
Results match: True



## Convolution with padding

Just add padding to the image, and use the previous function 

In [18]:
def conv_image_with_padding(image, kernel, padding=0):
    """
    Convolution on a 2D image with padding
    No stride (stride=1)
    
    Args:
        image: 2D numpy array (height, width)
        kernel: 2D numpy array 
        padding: int (padding size)
    
    Returns:
        2D numpy array (convolved output)
    """
    # Add padding to image
    if padding > 0:
        padded_image = np.pad(image, padding, mode='constant', constant_values=0)
    else:
        padded_image = image
    
    # Now use our level 2 function on padded image
    return conv_image_basic(padded_image, kernel)

In [19]:
# Create a simple 4x4 image
image = np.array([[1, 2, 3, 4],
                 [5, 6, 7, 8],
                 [9, 10, 11, 12],
                 [13, 14, 15, 16]])

# 3x3 kernel
kernel = np.array([[1, 0, -1],
                  [2, 0, -2],
                  [1, 0, -1]])  # Sobel X

print("Input image (4x4):")
print(image)
print(f"\nKernel (3x3):")
print(kernel)

# Test different padding values
for pad in [0, 1, 2]:
    result = conv_image_with_padding(image, kernel, padding=pad)
    
    print(f"\nWith padding={pad}:")
    if pad > 0:
        padded = np.pad(image, pad, mode='constant', constant_values=0)
        print(f"Padded image shape: {padded.shape}")
    print(f"Output shape: {result.shape}")
    print("Output:")
    print(result)
    
    # Verify with PyTorch
    image_torch = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)
    kernel_torch = torch.from_numpy(kernel).float().unsqueeze(0).unsqueeze(0)
    torch_result = F.conv2d(image_torch, kernel_torch, padding=pad).squeeze().numpy()
    
    print(f"PyTorch matches: {np.allclose(result, torch_result)}")
print()

Input image (4x4):
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]

Kernel (3x3):
[[ 1  0 -1]
 [ 2  0 -2]
 [ 1  0 -1]]

With padding=0:
Output shape: (2, 2)
Output:
[[-8. -8.]
 [-8. -8.]]
PyTorch matches: True

With padding=1:
Padded image shape: (6, 6)
Output shape: (4, 4)
Output:
[[-10.  -6.  -6.  13.]
 [-24.  -8.  -8.  28.]
 [-40.  -8.  -8.  44.]
 [-38.  -6.  -6.  41.]]
PyTorch matches: True

With padding=2:
Padded image shape: (8, 8)
Output shape: (6, 6)
Output:
[[ -1.  -2.  -2.  -2.   3.   4.]
 [ -7. -10.  -6.  -6.  13.  16.]
 [-20. -24.  -8.  -8.  28.  32.]
 [-36. -40.  -8.  -8.  44.  48.]
 [-35. -38.  -6.  -6.  41.  44.]
 [-13. -14.  -2.  -2.  15.  16.]]
PyTorch matches: True



## Convlution with stride and padding 

In [22]:
def conv_image_full(image, kernel, stride=1, padding=0):
    """
    Full convolution implementation with stride and padding
    
    Args:
        image: 2D numpy array (height, width)
        kernel: 2D numpy array
        stride: int (stride size)
        padding: int (padding size)
    
    Returns:
        2D numpy array (convolved output)
    """
    # Add padding to image
    if padding > 0:
        padded_image = np.pad(image, padding, mode='constant', constant_values=0)
    else:
        padded_image = image
    
    img_h, img_w = padded_image.shape
    ker_h, ker_w = kernel.shape
    
    # Calculate output dimensions with stride
    # padding information has already been added 
    out_h = (img_h - ker_h) // stride + 1
    out_w = (img_w - ker_w) // stride + 1
    
    # Initialize output
    output = np.zeros((out_h, out_w))
    
    # Slide kernel over image with stride
    for i in range(out_h):
        for j in range(out_w):
            # Calculate position with stride
            # there's also a different way to implement this in for loop
            start_i = i * stride
            start_j = j * stride
            
            # Extract patch from image
            patch = padded_image[start_i:start_i+ker_h, start_j:start_j+ker_w]
            # Use our basic convolution function
            output[i, j] = conv_basic(patch, kernel)
    
    return output


In [24]:
# Create a 6x6 image
image = np.random.randint(0, 10, (6, 6))

# 3x3 kernel
kernel = np.array([[1, 1, 1],
                  [0, 0, 0],
                  [-1, -1, -1]])

print("Input image (6x6):")
print(image)
print(f"\nKernel (3x3):")
print(kernel)

# Test different combinations
test_configs = [
    {"stride": 1, "padding": 0},
    {"stride": 1, "padding": 1},
    {"stride": 2, "padding": 0},
    {"stride": 2, "padding": 1},
]

for config in test_configs:
    stride = config["stride"]
    padding = config["padding"]
    
    result = conv_image_full(image, kernel, stride=stride, padding=padding)
    
    print(f"\nStride={stride}, Padding={padding}:")
    print(f"Output shape: {result.shape}")
    print("Output:")
    print(result)
    
    # Verify with PyTorch
    image_torch = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)
    kernel_torch = torch.from_numpy(kernel).float().unsqueeze(0).unsqueeze(0)
    torch_result = F.conv2d(image_torch, kernel_torch, 
                           stride=stride, padding=padding).squeeze().numpy()
    
    print(f"PyTorch matches: {np.allclose(result, torch_result)}")
print()

Input image (6x6):
[[4 6 3 7 7 7]
 [9 0 5 3 3 7]
 [5 8 3 9 4 2]
 [8 7 9 4 7 3]
 [1 2 7 7 0 8]
 [9 2 1 8 0 1]]

Kernel (3x3):
[[ 1  1  1]
 [ 0  0  0]
 [-1 -1 -1]]

Stride=1, Padding=0:
Output shape: (4, 4)
Output:
[[ -3.  -4.   1.   6.]
 [-10. -12.  -9.  -1.]
 [  6.   4.   2.   0.]
 [ 12.   9.  11.   5.]]
PyTorch matches: True

Stride=1, Padding=1:
Output shape: (6, 6)
Output:
[[ -9. -14.  -8. -11. -13. -10.]
 [ -3.  -3.  -4.   1.   6.   8.]
 [ -6. -10. -12.  -9.  -1.   0.]
 [ 10.   6.   4.   2.   0.  -2.]
 [  4.  12.   9.  11.   5.   9.]
 [  3.  10.  16.  14.  15.   8.]]
PyTorch matches: True

Stride=2, Padding=0:
Output shape: (2, 2)
Output:
[[-3.  1.]
 [ 6.  2.]]
PyTorch matches: True

Stride=2, Padding=1:
Output shape: (3, 3)
Output:
[[ -9.  -8. -13.]
 [ -6. -12.  -1.]
 [  4.   9.   5.]]
PyTorch matches: True



## Multi Channel Convolution 

In [29]:
def conv2d_full(input_tensor, kernel, bias=None, stride=1, padding=0):
    """
    Full 2D convolution with multiple channels, batches, stride, and padding
    
    Args:
        input_tensor: 4D numpy array (batch_size, in_channels, height, width)
        kernel: 4D numpy array (out_channels, in_channels, kernel_height, kernel_width)
        bias: 1D numpy array (out_channels,) or None
        stride: int
        padding: int
    
    Returns:
        4D numpy array (batch_size, out_channels, out_height, out_width)
    """
    batch_size, in_channels, input_h, input_w = input_tensor.shape
    out_channels, _, kernel_h, kernel_w = kernel.shape
    
    # Add padding to input
    if padding > 0:
        padded_input = np.pad(input_tensor, 
                             ((0, 0), (0, 0), (padding, padding), (padding, padding)),
                             mode='constant', constant_values=0)
    else:
        padded_input = input_tensor
    
    _, _, padded_h, padded_w = padded_input.shape
    
    # Calculate output dimensions
    output_h = (padded_h - kernel_h) // stride + 1
    output_w = (padded_w - kernel_w) // stride + 1
    
    # Initialize output tensor
    output = np.zeros((batch_size, out_channels, output_h, output_w))
    
    # Perform convolution
    for b in range(batch_size):
        for oc in range(out_channels):
            for oh in range(output_h):
                for ow in range(output_w):
                    # Calculate input region with stride
                    h_start = oh * stride
                    h_end = h_start + kernel_h
                    w_start = ow * stride
                    w_end = w_start + kernel_w
                    
                    # Extract input patch (all input channels)
                    input_patch = padded_input[b, :, h_start:h_end, w_start:w_end]
                    
                    # Compute convolution: sum over all input channels
                    conv_result = 0
                    for ic in range(in_channels):
                        # Use our basic convolution function for each channel
                        conv_result += conv_basic(input_patch[ic], kernel[oc, ic])
                    
                    output[b, oc, oh, ow] = conv_result
                    
                    # Add bias if provided
                    if bias is not None:
                        output[b, oc, oh, ow] += bias[oc]
    
    return output

In [35]:
# to get good results, I had to round off some values. 

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Create test data
batch_size, in_channels, height, width = 2, 3, 6, 6
out_channels, kernel_h, kernel_w = 2, 3, 3

input_tensor = np.random.randn(batch_size, in_channels, height, width).astype(np.float32)
kernel = np.random.randn(out_channels, in_channels, kernel_h, kernel_w).astype(np.float32)
bias = np.random.randn(out_channels).astype(np.float32)

print(f"Input shape: {input_tensor.shape}")
print(f"Kernel shape: {kernel.shape}")
print(f"Bias shape: {bias.shape}")

# Test our implementation
result = conv2d_full(input_tensor, kernel, bias, stride=1, padding=1)

# Compare with PyTorch
input_torch = torch.from_numpy(input_tensor)
kernel_torch = torch.from_numpy(kernel)
bias_torch = torch.from_numpy(bias)
torch_result = F.conv2d(input_torch, kernel_torch, bias_torch, 
                       stride=1, padding=1).numpy()

print(f"\nOur result shape: {result.shape}")
print(f"PyTorch result shape: {torch_result.shape}")

# Round both results to handle floating point precision issues
result_rounded = np.round(result, decimals=6)
torch_result_rounded = np.round(torch_result, decimals=6)

# Calculate difference on rounded values
max_diff = np.max(np.abs(result_rounded - torch_result_rounded))
results_match = np.allclose(result_rounded, torch_result_rounded, rtol=1e-6, atol=1e-6)

print(f"Max difference (after rounding to 6 decimals): {max_diff:.2e}")
print(f"Results match (after rounding): {results_match}")

# Also show raw comparison for reference
raw_max_diff = np.max(np.abs(result - torch_result))
print(f"Raw max difference (before rounding): {raw_max_diff:.2e}")
print(f"Raw results match: {np.allclose(result, torch_result, rtol=1e-5, atol=1e-6)}")

Input shape: (2, 3, 6, 6)
Kernel shape: (2, 3, 3, 3)
Bias shape: (2,)

Our result shape: (2, 2, 6, 6)
PyTorch result shape: (2, 2, 6, 6)
Max difference (after rounding to 6 decimals): 2.33e-06
Results match (after rounding): True
Raw max difference (before rounding): 2.26e-06
Raw results match: True


# Maxpooling from scratch 

In [36]:
def maxpool_basic(input_array):
    """
    Most basic max pooling: find maximum value in an array
    
    Args:
        input_array: 2D numpy array
    
    Returns:
        Single scalar value (maximum in the array)
    """
    return np.max(input_array)

In [38]:
def maxpool_image_basic(image, pool_size):
    """
    Max pooling on a 2D image with a pooling window
    Non-overlapping pooling (stride = pool_size)
    
    Args:
        image: 2D numpy array (height, width)
        pool_size: int or tuple (pooling window size)
    
    Returns:
        2D numpy array (max pooled output)
    """
    if isinstance(pool_size, int):
        pool_h, pool_w = pool_size, pool_size
    else:
        pool_h, pool_w = pool_size
    
    img_h, img_w = image.shape
    
    # Calculate output dimensions (non-overlapping pooling)
    out_h = img_h // pool_h
    out_w = img_w // pool_w
    
    # Initialize output
    output = np.zeros((out_h, out_w))
    
    # Apply max pooling
    for i in range(out_h):
        for j in range(out_w):
            # Extract patch from image (non-overlapping)
            start_i = i * pool_h
            start_j = j * pool_w
            patch = image[start_i:start_i+pool_h, start_j:start_j+pool_w]
            # Use our basic max function
            output[i, j] = maxpool_basic(patch)
    
    return output

In [40]:

# Create a simple 4x4 image
image = np.array([[1, 2, 3, 4],
                 [5, 6, 7, 8],
                 [9, 10, 11, 12],
                 [13, 14, 15, 16]])

print("Input image (4x4):")
print(image)

# Test 2x2 pooling
result_2x2 = maxpool_image_basic(image, pool_size=2)
print(f"\nPool size: 2x2")
print(f"Output shape: {result_2x2.shape} (expected: {(4//2, 4//2)})")
print("Output:")
print(result_2x2)

# Verify with PyTorch
image_torch = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)
torch_result = F.max_pool2d(image_torch, kernel_size=2, stride=2).squeeze().numpy()

print(f"\nPyTorch result:")
print(torch_result)
print(f"Results match: {np.allclose(result_2x2, torch_result)}")

# Test with 6x6 image and different pool sizes
image_6x6 = np.random.randint(1, 20, (6, 6))
print(f"\n\nTest with 6x6 image:")
print(image_6x6)

for pool_size in [2, 3]:
    result = maxpool_image_basic(image_6x6, pool_size)
    print(f"\nPool size: {pool_size}x{pool_size}")
    print(f"Output shape: {result.shape}")
    print("Output:")
    print(result)
print()

Input image (4x4):
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]

Pool size: 2x2
Output shape: (2, 2) (expected: (2, 2))
Output:
[[ 6.  8.]
 [14. 16.]]

PyTorch result:
[[ 6.  8.]
 [14. 16.]]
Results match: True


Test with 6x6 image:
[[ 9  7  4 18 13 11]
 [ 4  4 10  5  9  3]
 [17  3 16  4 18 17]
 [ 7  5 12 17 13  3]
 [ 9 17 17 16 13 19]
 [17  4 12  9 19 12]]

Pool size: 2x2
Output shape: (3, 3)
Output:
[[ 9. 18. 13.]
 [17. 17. 18.]
 [17. 17. 19.]]

Pool size: 3x3
Output shape: (2, 2)
Output:
[[17. 18.]
 [17. 19.]]



In [44]:
def maxpool_image_with_stride(image, pool_size, stride=None, padding=0):
    """
    Max pooling with custom stride
    
    Args:
        image: 2D numpy array (height, width)
        pool_size: int or tuple (pooling window size)
        stride: int or tuple (stride size). If None, defaults to pool_size
        padding: int (padding size)
    
    Returns:
        2D numpy array (max pooled output)
    """
    if isinstance(pool_size, int):
        pool_h, pool_w = pool_size, pool_size
    else:
        pool_h, pool_w = pool_size
    
    if stride is None:
        stride_h, stride_w = pool_h, pool_w
    elif isinstance(stride, int):
        stride_h, stride_w = stride, stride
    else:
        stride_h, stride_w = stride
    
    # Add padding to image
    if padding > 0:
        padded_image = np.pad(image, padding, mode='constant', constant_values=0) # could be an issue if image has negative values 
    else:
        padded_image = image
    
    img_h, img_w = padded_image.shape
    
    # Calculate output dimensions with custom stride
    out_h = (img_h - pool_h) // stride_h + 1
    out_w = (img_w - pool_w) // stride_w + 1
    
    # Initialize output
    output = np.zeros((out_h, out_w))
    
    # Apply max pooling with stride
    for i in range(out_h):
        for j in range(out_w):
            # Calculate position with stride
            start_i = i * stride_h
            start_j = j * stride_w
            
            # Extract patch from image
            patch = padded_image[start_i:start_i+pool_h, start_j:start_j+pool_w]
            # Use our basic max function
            output[i, j] = maxpool_basic(patch)
    
    return output

In [45]:
"""Test max pooling with custom stride"""
print("MAXPOOL LEVEL 4: MaxPool with Custom Stride")
print("=" * 60)

# Create a 6x6 image
image = np.random.randint(1, 20, (6, 6))

print("Input image (6x6):")
print(image)

# Test different combinations
test_configs = [
    {"pool_size": 2, "stride": 1, "padding": 0},
    {"pool_size": 2, "stride": 2, "padding": 0},  # Non-overlapping
    {"pool_size": 3, "stride": 1, "padding": 0},  # Overlapping
    {"pool_size": 3, "stride": 2, "padding": 1},
]

for config in test_configs:
    pool_size = config["pool_size"]
    stride = config["stride"]
    padding = config["padding"]
    
    result = maxpool_image_with_stride(image, pool_size=pool_size, 
                                     stride=stride, padding=padding)
    
    print(f"\nPool_size={pool_size}, Stride={stride}, Padding={padding}:")
    print(f"Output shape: {result.shape}")
    print("Output:")
    print(result)
    
    # Verify with PyTorch
    image_torch = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)
    torch_result = F.max_pool2d(image_torch, kernel_size=pool_size, 
                               stride=stride, padding=padding).squeeze().numpy()
    
    print(f"PyTorch result:")
    print(torch_result)
    print(f"PyTorch matches: {np.allclose(result, torch_result)}")
print()

MAXPOOL LEVEL 4: MaxPool with Custom Stride
Input image (6x6):
[[ 5  5  6 19  8 16]
 [13  1 17  7 13  4]
 [ 4  6 19 12  7 10]
 [19  7  3 13 13 18]
 [ 8  9  7  1  3 13]
 [17  1  6  6 12 13]]

Pool_size=2, Stride=1, Padding=0:
Output shape: (5, 5)
Output:
[[13. 17. 19. 19. 16.]
 [13. 19. 19. 13. 13.]
 [19. 19. 19. 13. 18.]
 [19.  9. 13. 13. 18.]
 [17.  9.  7. 12. 13.]]
PyTorch result:
[[13. 17. 19. 19. 16.]
 [13. 19. 19. 13. 13.]
 [19. 19. 19. 13. 18.]
 [19.  9. 13. 13. 18.]
 [17.  9.  7. 12. 13.]]
PyTorch matches: True

Pool_size=2, Stride=2, Padding=0:
Output shape: (3, 3)
Output:
[[13. 19. 16.]
 [19. 19. 18.]
 [17.  7. 13.]]
PyTorch result:
[[13. 19. 16.]
 [19. 19. 18.]
 [17.  7. 13.]]
PyTorch matches: True

Pool_size=3, Stride=1, Padding=0:
Output shape: (4, 4)
Output:
[[19. 19. 19. 19.]
 [19. 19. 19. 18.]
 [19. 19. 19. 18.]
 [19. 13. 13. 18.]]
PyTorch result:
[[19. 19. 19. 19.]
 [19. 19. 19. 18.]
 [19. 19. 19. 18.]
 [19. 13. 13. 18.]]
PyTorch matches: True

Pool_size=3, Stride=2, Pa

In [46]:
def maxpool2d_full(input_tensor, pool_size, stride=None, padding=0):
    """
    Full 2D max pooling with multiple channels and batches
    
    Args:
        input_tensor: 4D numpy array (batch_size, channels, height, width)
        pool_size: int or tuple (pooling window size)
        stride: int, tuple, or None (stride size)
        padding: int (padding size)
    
    Returns:
        4D numpy array (batch_size, channels, out_height, out_width)
    """
    if isinstance(pool_size, int):
        pool_h, pool_w = pool_size, pool_size
    else:
        pool_h, pool_w = pool_size
    
    if stride is None:
        stride_h, stride_w = pool_h, pool_w
    elif isinstance(stride, int):
        stride_h, stride_w = stride, stride
    else:
        stride_h, stride_w = stride
    
    batch_size, channels, input_h, input_w = input_tensor.shape
    
    # Add padding to input
    if padding > 0:
        padded_input = np.pad(input_tensor,
                             ((0, 0), (0, 0), (padding, padding), (padding, padding)),
                             mode='constant', constant_values=-np.inf)
    else:
        padded_input = input_tensor
    
    _, _, padded_h, padded_w = padded_input.shape
    
    # Calculate output dimensions
    output_h = (padded_h - pool_h) // stride_h + 1
    output_w = (padded_w - pool_w) // stride_w + 1
    
    # Initialize output tensor
    output = np.zeros((batch_size, channels, output_h, output_w))
    
    # Perform max pooling
    for b in range(batch_size):
        for c in range(channels):
            for oh in range(output_h):
                for ow in range(output_w):
                    # Calculate input region with stride
                    h_start = oh * stride_h
                    h_end = h_start + pool_h
                    w_start = ow * stride_w
                    w_end = w_start + pool_w
                    
                    # Extract input patch for this channel
                    input_patch = padded_input[b, c, h_start:h_end, w_start:w_end]
                    
                    # Use our basic max function
                    output[b, c, oh, ow] = maxpool_basic(input_patch)
    
    return output

In [49]:
"""Test full multi-channel max pooling"""
print("MAXPOOL LEVEL 5: Full Multi-Channel MaxPool")
print("=" * 60)

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Create test data
batch_size, channels, height, width = 2, 3, 8, 8

input_tensor = np.random.randn(batch_size, channels, height, width).astype(np.float32)

print(f"Input shape: {input_tensor.shape}")

# Test different configurations
test_configs = [
    {"pool_size": 2, "stride": 2, "padding": 0, "name": "2x2 pool, stride=2, no padding"},
    {"pool_size": 2, "stride": 1, "padding": 0, "name": "2x2 pool, stride=1, no padding"},  
    {"pool_size": 3, "stride": 2, "padding": 1, "name": "3x3 pool, stride=2, padding=1"},
]

for config in test_configs:
    pool_size = config["pool_size"]
    stride = config["stride"]
    padding = config["padding"]
    name = config["name"]
    
    print(f"\nTesting: {name}")
    print("-" * 40)
    
    # Test our implementation
    result = maxpool2d_full(input_tensor, pool_size=pool_size, 
                           stride=stride, padding=padding)
    
    # Compare with PyTorch
    input_torch = torch.from_numpy(input_tensor)
    torch_result = F.max_pool2d(input_torch, kernel_size=pool_size,
                               stride=stride, padding=padding).numpy()
    
    print(f"Our result shape: {result.shape}")
    print(f"PyTorch result shape: {torch_result.shape}")
    
    # Round both results to handle floating point precision issues
    result_rounded = np.round(result, decimals=6)
    torch_result_rounded = np.round(torch_result, decimals=6)
    
    # Calculate difference on rounded values
    max_diff = np.max(np.abs(result_rounded - torch_result_rounded))
    results_match = np.allclose(result_rounded, torch_result_rounded, rtol=1e-6, atol=1e-6)
    
    print(f"Max difference (after rounding to 6 decimals): {max_diff:.2e}")
    print(f"Results match (after rounding): {results_match}")
    
    # Also show raw comparison for reference
    raw_max_diff = np.max(np.abs(result - torch_result))
    print(f"Raw max difference (before rounding): {raw_max_diff:.2e}")
    print(f"Raw results match: {np.allclose(result, torch_result, rtol=1e-5, atol=1e-6)}")


MAXPOOL LEVEL 5: Full Multi-Channel MaxPool
Input shape: (2, 3, 8, 8)

Testing: 2x2 pool, stride=2, no padding
----------------------------------------
Our result shape: (2, 3, 4, 4)
PyTorch result shape: (2, 3, 4, 4)
Max difference (after rounding to 6 decimals): 1.03e-06
Results match (after rounding): True
Raw max difference (before rounding): 0.00e+00
Raw results match: True

Testing: 2x2 pool, stride=1, no padding
----------------------------------------
Our result shape: (2, 3, 7, 7)
PyTorch result shape: (2, 3, 7, 7)
Max difference (after rounding to 6 decimals): 1.03e-06
Results match (after rounding): True
Raw max difference (before rounding): 0.00e+00
Raw results match: True

Testing: 3x3 pool, stride=2, padding=1
----------------------------------------
Our result shape: (2, 3, 4, 4)
PyTorch result shape: (2, 3, 4, 4)
Max difference (after rounding to 6 decimals): 1.03e-06
Results match (after rounding): True
Raw max difference (before rounding): 0.00e+00
Raw results match: 